In [96]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import json
import pandas as pd

In [97]:
options = Options()
options.headless = False
options.add_argument("--window-size=1920,1080")
service = Service(executable_path='./chromedriver.exe')

In [98]:
news_dict = {'category':[], 'title': [], 'date': [], 'content': [], 'sources':[]}

# Crawling Process
1. get url about agricultural product
    - [RIce](http://agro.gov.vn/vn/xc9_Lua-gao.html)
    - [Coffee](http://agro.gov.vn/vn/xc2_Ca-phe.html)
2. from url we got from (1), access link to get title, date and content news
3. save to jsonl and csv

## Get Url

In [99]:
for i in range(1,361):
    browser = webdriver.Chrome(service=service, options=options)
    browser.get(f"https://agro.gov.vn/vn/p{i}_9Lua-gao.html")
    elements = browser.find_elements(By.CSS_SELECTOR, ".news [href]")
    links = [element.get_attribute('href') for element in elements]
    browser.quit()
    news_dict['sources'] += [*set(links[0:10])]

## Get Title, Content, Data

In [ ]:
for i in range(len(news_dict['sources'])):
    browser = webdriver.Chrome(service=service, options=options)
    url = news_dict['sources'][i]
    browser.get(url)
    sleep(1)
    try:
        #Title
        element = browser.find_element(By.ID, "ctl00_maincontent_N_TIEUDE")
        news_dict['title'].append(element.text)
        #Date
        element = browser.find_element(By.ID, "ctl00_maincontent_N_NGAYTHANG")
        news_dict['date'].append(element.text)
        #Content
        element = browser.find_element(By.ID, "ctl00_maincontent_N_NOIDUNG")
        news_dict['content'].append(element.text)

    except:
        news_dict['title'].append(" ")
        news_dict['date'].append(" ")
        news_dict['content'].append(" ")
    browser.quit()

In [ ]:
pd.DataFrame.from_dict(news_dict).to_csv('data/news.csv')

['https://agro.gov.vn/vn/tID32537_Xuat-khau-gao-8-thang-dat-muc-ky-luc-.html',
 'https://agro.gov.vn/vn/tID32438_Philippines-dam-phan-mua-gao-Viet-Nam-an-Do-.html',
 'https://agro.gov.vn/vn/tID32420_Gia-gao-tang-nhanh-gioi-buon-than-trong-nhap-hang.html',
 'https://agro.gov.vn/vn/tID32502_-Viet-Nam-dang-xuat-nhieu-gao-nhat-the-gioi-.html',
 'https://agro.gov.vn/vn/tID32476_Giu-thuong-hieu-gao-Viet-giua-bien-dong-luong-thuc-toan-cau.html',
 'https://agro.gov.vn/vn/tID32407_Gia-lua-gao-len-cao-Thu-tuong-ra-chi-thi-nong.html',
 'https://agro.gov.vn/vn/tID32480_an-Do-siet-them-nguon-cung-gao-toan-cau-.html',
 'https://agro.gov.vn/vn/tID32448_Bo-Cong-Thuong-ban-hanh-Chi-thi-ve-xuat-khau-gao-va-binh-on-thi-truong-trong-nuoc.html',
 'https://agro.gov.vn/vn/tID32604_-Thi-truong-tram-lang-gia-gao-giam-nhiet-.html',
 'https://agro.gov.vn/vn/tID32576_-Thi-truong-xuat-khau-gao-con-nhieu-bien-dong-.html',
 'https://agro.gov.vn/vn/tID32365_Sau-an-Do-them-Nga-va-UAE-cam-xuat-khau-gao.html',
 'https:/